In [4]:
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import pandas as pd
import numpy as np
from classic_patterns import ml_tsla_data
from labeling import label_tsla_data

## Juntando os dados
Pegando os dados

In [ ]:
ml_tsla_data = np.array(ml_tsla_data)  
label_tsla_data = np.array(label_tsla_data).ravel()  

# Divide os dados em conjuntos de treinamento e teste
X_train, X_test, y_train, y_test = train_test_split(ml_tsla_data, label_tsla_data, test_size=0.2, random_state=42)

## Fazendo a rede neural

In [8]:
# Cria a rede neural
mlp = MLPClassifier(hidden_layer_sizes=(100,), activation='logistic', solver='adam', max_iter=500, random_state=42)

# Treina a rede neural
mlp.fit(X_train, y_train)

# Faz previsões
y_pred = mlp.predict(X_test)

# Avalia o modelo
print(classification_report(y_test, y_pred, target_names=["0", "1", "-1"]))


              precision    recall  f1-score   support

           0       0.00      0.00      0.00         9
           1       0.00      0.00      0.00       357
          -1       0.49      0.99      0.65       347

    accuracy                           0.48       713
   macro avg       0.16      0.33      0.22       713
weighted avg       0.24      0.48      0.32       713



C:\Users\kauan\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\kauan\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\kauan\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\metrics\_classificati